<a href="https://colab.research.google.com/github/MrSuicideParrot/HELK/blob/master/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy import spatial
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive
import math
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Variaveis globais
global vocab_size_text
global vocab_size_title
global MAX_LENGTH_TEXT
global MAX_LENGTH_TITLE
global mymodel
global history
global covid_title

def readJson(dt):
  title = [t[0] for t in list(dt.values())]
  text = [t[1] for t in list(dt.values())]
  dtk = list(dt.keys())
  return pd.DataFrame({'url': dtk, 'title': title, 'text': text})

def read(data, json = True):
  if json:
    df = readJson(data)
  else:
    df = pd.read_csv(data)
  df.dropna(inplace=True)
  return df 

def tokenData(cdata):
  cdata = cdata.astype(str)
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(cdata)
  word_index = tokenizer.word_index
  return tokenizer.texts_to_sequences(cdata), word_index

def setVariables(ctext, ctitle):
  X_lens = [len(x) for x in ctext.values]
  X_lens = np.array(X_lens)

  X_lens1 = [len(x) for x in ctitle.values]
  X_lens1 = np.array(X_lens1)

  m = math.ceil(np.mean(X_lens))
  std = math.ceil(np.std(X_lens))
  global MAX_LENGTH_TEXT
  MAX_LENGTH_TEXT = m + 2*std

  m1 = math.ceil(np.mean(X_lens1))
  std1 = math.ceil(np.std(X_lens1))
  global MAX_LENGTH_TITLE
  MAX_LENGTH_TITLE = m1 + 2*std1

def splitData(ctext, ctitle, clabel):
  split = 0.2
  split_n = int(round(len(ctext)*(1-split),0))

  train_text = ctext[split_n:]
  train_title = ctitle[split_n:]
  train_labels = clabel[split_n:]

  test_text = ctext[:split_n]
  test_title = ctitle[:split_n]
  test_labels = clabel[:split_n]

  setVariables(ctext, ctitle)
  
  return train_text, train_title, train_labels, test_text, test_title, test_labels 

def padding(cdata, title = True): 
  if title:
    maxVar = MAX_LENGTH_TITLE
  else:
    maxVar = MAX_LENGTH_TEXT
    
   
  return tf.keras.preprocessing.sequence.pad_sequences(cdata,
                                              maxlen=maxVar,
                                              padding='post',
                                              truncating='post')
def embedding(word_index, title = True):
  fileGlove = '/content/drive/My Drive/fake-news/glove.6B.100d.txt'
  embeddings_dict = {}
  with open(fileGlove, 'r', encoding="utf8") as f:
    for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      embeddings_dict[word] = vector
  
  if title:
    embeddings_matrix = np.zeros((vocab_size_title + 1, 100))
  else:
    embeddings_matrix = np.zeros((vocab_size_text + 1, 100))
    
  for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
      embeddings_matrix[i] = embedding_vector
  return embeddings_matrix

def defineModel(emb_title, emb_text, train_text, train_title, train_labels, test_text, test_title, test_labels):
  text_input = tf.keras.layers.Input(shape=(MAX_LENGTH_TEXT,))
  text_embed = tf.keras.layers.Embedding(vocab_size_text + 1, 100, weights=[emb_text], trainable=False)(text_input)
  text_drop = tf.keras.layers.Dropout(0.2)(text_embed)
  text_conv = tf.keras.layers.Conv1D(64, 5, activation='relu')(text_drop)
  text_pool = tf.keras.layers.MaxPooling1D(pool_size=4)(text_conv)
  text_lstm1 = tf.keras.layers.LSTM(20, return_sequences=True)(text_pool)
  text_lstm2 = tf.keras.layers.LSTM(20)(text_lstm1)

  title_input = tf.keras.layers.Input(shape=(MAX_LENGTH_TITLE,))
  title_embed = tf.keras.layers.Embedding(vocab_size_title + 1, 100, weights=[emb_title], trainable=False)(title_input)
  title_drop = tf.keras.layers.Dropout(0.2)(title_embed)
  title_conv = tf.keras.layers.Conv1D(64, 5, activation='relu')(title_drop)
  title_pool = tf.keras.layers.MaxPooling1D(pool_size=4)(title_conv)
  title_lstm1 = tf.keras.layers.LSTM(20, return_sequences=True)(title_pool)
  title_lstm2 = tf.keras.layers.LSTM(20)(title_lstm1)

  concat = tf.keras.layers.concatenate([text_lstm2, title_lstm2])

  drop = tf.keras.layers.Dropout(0.2)(concat)
  dense1 = tf.keras.layers.Dense(100)(drop)
  drop1 = tf.keras.layers.Dropout(0.3)(dense1)
  dense2 = tf.keras.layers.Dense(50)(drop1)
  output = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

  model = tf.keras.models.Model(inputs=[text_input, title_input], outputs=output)
  #model.summary()

  METRICS = [
        tf.metrics.Precision(name='precision'),
        tf.metrics.Recall(name='recall'),
        tf.metrics.AUC(name='auc'),
  ]

  model.compile(optimizer=tf.keras.optimizers.Adam(0.0005),
                loss='binary_crossentropy',
                metrics=METRICS)
  
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc',
                                            patience=5,
                                            mode='max',
                                          restore_best_weights=True)
  history = model.fit([train_text, train_title],
                     train_labels,
                     epochs=5,
                     batch_size=100,
                     validation_data=(
                           [test_text, test_title],
                            test_labels
                     ),
  callbacks=[callback])

  return model, history

def prediction(test):
  preds = mymodel.predict(test)
  return [p[0]for p in preds]

def covidNews(path):
  covid = read(path, json = False)
  covid = covid[:500]
  covid['title'],_ = tokenData(covid['title'])
  global covid_title
  covid_title = padding(covid['title'], title = True)

def avg(x):
  return np.mean(list(filter(lambda y : y > 0, x)))

def comparisons(test_title):
    return [avg(list(map(lambda y: spatial.distance.cosine(x, y), covid_title))) for x in test_title]

def trainModel(path, debug = False):
  df = read(path, json = False)

  if debug:
    print("Read data frame")

  df['title'], word_index_title = tokenData(df['title'])
  global vocab_size_title
  vocab_size_title = len(word_index_title)
  df['text'], word_index_text = tokenData(df['text'])
  global vocab_size_text
  vocab_size_text = len(word_index_text)

  if debug:
    print("Tokenizer complete")

  train_text, train_title, train_labels, test_text, test_title, test_labels = splitData(df['text'], df['title'], df['label'])

  train_text = padding(train_text, title = False)
  train_title = padding(train_title, title = True)
  test_title = padding(test_title, title = True)
  test_text = padding(test_text, title = False)
  
  if debug:
    print("Split and padding done")

  embeddings_matrix_title = embedding(word_index_title, title = True)
  embeddings_matrix_text = embedding(word_index_text, title = False)

  if debug:
    print("Embeddings all set")

  global mymodel, history
  mymodel, history = defineModel(embeddings_matrix_title, embeddings_matrix_text, train_text, train_title, train_labels, test_text, test_title, test_labels)
  
  if debug:
    print("Model compiled")
  
  !mkdir -p /content/drive/My\ Drive/saved_model
  covidNews("/content/drive/My Drive/fake-news/corona_dataset.csv")
  np.save('/content/drive/My Drive/saved_model/covid_title.npy', covid_title)

  if debug:
    print("Covid comparisons")

  mymodel.save('/content/drive/My Drive/saved_model/my_model')

def testModel(path, debug = False):
  try:
    mymodel
  except NameError:
    global model
    mymodel = tf.keras.models.load_model('/content/drive/My Drive/saved_model/my_model')
    global covid_title
    covid_title = np.load('/content/drive/My Drive/saved_model/covid_title.npy')
  
  df = read(path, json = True)
  
  if debug:
    print("Data frame read from json")

  df['title'], word_index_title = tokenData(df['title'])
  global vocab_size_title
  vocab_size_title = len(word_index_title)
  df['text'], word_index_text = tokenData(df['text'])
  global vocab_size_text
  vocab_size_text = len(word_index_text)
  
  if debug:
    print("Tokenizer complete")

  test_title = padding(df['title'], title = True)
  test_text = padding(df['text'], title = False)

  if debug:
    print("Padding done")
  
  preds = prediction([test_text, test_title])

  if debug:
    print("Predictions")

  t = comparisons(test_title)

  data = {}
  data['url'] = df['url']
  data['predictions'] = preds
  data['compare'] = t
  
  return data

In [3]:
export = trainModel(path = '/content/drive/My Drive/fake-news/train.csv', debug = True)

Read data frame
Tokenizer complete
Split and padding done
Embeddings all set
Epoch 1/5
37/37 [==============================] - 11s 306ms/step - loss: 0.6696 - precision: 0.5217 - recall: 0.0961 - auc: 0.6243 - val_loss: 0.6158 - val_precision: 0.8825 - val_recall: 0.2254 - val_auc: 0.8938
Epoch 2/5
37/37 [==============================] - 10s 265ms/step - loss: 0.4815 - precision: 0.7864 - recall: 0.7123 - auc: 0.8815 - val_loss: 0.3395 - val_precision: 0.7711 - val_recall: 0.9532 - val_auc: 0.9268
Epoch 3/5
37/37 [==============================] - 10s 263ms/step - loss: 0.2970 - precision: 0.8155 - recall: 0.9316 - auc: 0.9354 - val_loss: 0.2613 - val_precision: 0.8199 - val_recall: 0.9552 - val_auc: 0.9470
Epoch 4/5
37/37 [==============================] - 10s 264ms/step - loss: 0.2547 - precision: 0.8330 - recall: 0.9464 - auc: 0.9508 - val_loss: 0.2440 - val_precision: 0.8353 - val_recall: 0.9379 - val_auc: 0.9538
Epoch 5/5
37/37 [==============================] - 10s 264ms/step -